In [1]:
# imports
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import re
import numpy as np

In [2]:
driver = webdriver.Chrome("/Applications/chromedriver")
url = "https://neos-guide.org/content/diet-problem-solver"
driver.get(url)
content = driver.page_source
soup = BeautifulSoup(content)

In [3]:
foods = pd.DataFrame()
table = soup.find('div', attrs={'id':'foods_selection'}).find('table').find('tbody')

headers = []
for header in table.findAll('tr', recursive=False)[1].findAll('td', recursive=False)[1:]:
    headers.append(header.findAll(text=True)[0])
    
for row in table.findAll('tr', recursive=False)[2:]:
    entries = []
    for cell in row.findAll('td', recursive=False)[1:]:
        if cell.find('input') is not None:
            entries.append(str(cell.find('input')).split('=')[-1].split('"')[1])
        else:
            entries.append((cell.findAll(text=True)[0]))
    foods = foods.append({headers[i] : entries[i] for i in range(5)}, ignore_index=True)
    
foods = foods[headers]

In [4]:
nutrients = pd.DataFrame()
table = soup.find('div', attrs={'id':'nutrients_selection'}).find('table').find('tbody')

headers = []
for header in table.findAll('tr', recursive=False)[1].findAll('td', recursive=False)[1:]:
    headers.append(header.findAll(text=True)[0])
    
for row in table.findAll('tr', recursive=False)[2:]:
    entries = []
    for cell in row.findAll('td', recursive=False)[1:]:
        if cell.find('input') is not None:
            entries.append(str(cell.find('input')).split('=')[-1].split('"')[1])
        else:
            entries.append((cell.findAll(text=True)[0]))
    nutrients = nutrients.append({headers[i] : entries[i] for i in range(4)}, ignore_index=True)
    
nutrients = nutrients[headers]

In [5]:
nutrients_info = pd.DataFrame()
table = soup.find('div', attrs={'id':'nutrient_info'}).find('table').find('tbody')
table
headers = []
for header in table.findAll('tr', recursive=False)[0].findAll('td', recursive=False):
    headers.append(header.findAll(text=True)[0])
    
for row in table.findAll('tr', recursive=False)[1:]:
    entries = []
    for cell in row.findAll('td', recursive=False):
        if len(cell.findAll(text=True)) != 0:
            entries.append((cell.findAll(text=True)[0]))
    if len(entries) == 12:
        nutrients_info = nutrients_info.append({headers[i] : entries[i] for i in range(12)}, ignore_index=True)
        
nutrients_info = nutrients_info[headers]

In [6]:
nutrients.head()

,Name,Unit,Min,Max
0,Calories,cal,2000,2250
1,Cholesterol,mg,0,300
2,Total_Fat,g,0,65
3,Sodium,mg,0,2400
4,Carbohydrates,g,0,300


In [7]:
foods = foods.merge(nutrients_info, on='Name', how='left').dropna()
foods.head()

,Name,Serving,Price/Serving ($),Min,Max,Calories,Cholesterol,Total_Fat,Sodium,Carbohydrates,Dietary_Fiber,Protein,Vit_A,Vit_C,Calcium,Iron
0,Frozen Broccoli,10 Oz Pkg,0.16,0,10,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots, Raw",1/2 Cup Shredded,0.07,0,10,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",1 Stalk,0.04,0,10,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,1/2 Cup,0.18,0,10,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce, Iceberg,Raw",1 Leaf,0.02,0,10,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [8]:
# export
foods.set_index('Name').rename(columns={'Price/Serving ($)' : 'Cost'}).to_csv('neos_foods.csv')
nutrients.set_index('Name').to_csv('neos_nutrients.csv')